# Zip Codes

In [ ]:
# Load Require Library
import pandas as pd
import re
import numpy as np

import time

from bs4 import BeautifulSoup
import requests
import re

import xlsxwriter

import itertools


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# for loop  & beautifulsoup to get the zip code info from the https://www.zipdatamaps.com/!

zipcodestable = pd.DataFrame()
counter = 0

for x in range (0,99999)   # We have zip codes, potentially,  between 1 to 99999.
    time.sleep(0.05)
    t = pd.Timestamp.now()
    x = str(x).zfill(5)
    
    zipurl= 'https://www.zipdatamaps.com/'+str(x)   # Website to use as a source!
    res_zip = requests.get(zipurl)
    # print(res_zip.status_code)
    zipcode_html_text = res_zip.text
    # print(zipcode_html_text)
    soup = BeautifulSoup(zipcode_html_text,'html.parser' )

    table = soup.find_all('table', attrs={"class": "table table-striped table-bordered table-hover table-condensed"})[0]       
    df = pd.read_html(str(table))[0]
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]
#     df

    title_text = soup.title
    for x in title_text:
         ZipCode = re.findall(r'Zip Code (\d{1,5})',x)

    df['zipcode'] = ZipCode
#     df
    print(counter, df['zipcode'], t, pd.Timestamp.now() - t)
    counter+=1
    zipcodestable = zipcodestable.append(df)

In [ ]:
df

In [ ]:
zipcodestable.to_csv('zipcodestable_test.csv', index = False)

In [ ]:
zipcodes2 = pd.read_csv(r'zipcodestable_test.csv', index_col = 'zipcode')
zipcodes2 = zipcodes2.rename(columns={'Coordinates(Y,X)':'Coordinates','Current Population:':'Current Pop','Primary County:':'County', 'Unemployment Rate:':'Unemployment_Rate','Racial Majority:':'Racial_Majority','School Test Performance:':'School_Test_Performance','Public School Racial Majority:':'Pub_School_Racial_Majority'})

In [ ]:
zipcodes2.dropna(subset = ['Official Zip Code Name'], inplace = True)
zipcodes2['Lat']  = zipcodes2['Coordinates'].str.extract('(\d{2,3}\.\d*)\,')
zipcodes2['Lan']  = zipcodes2['Coordinates'].str.extract('\,\s?(\-\d{2,3}\.\d*)')
zipcodes2['Area(Sqm)'] = zipcodes2['Area'].str.extract(r'(\d*)')
zipcodes2['Area(Sqm)'] = zipcodes2['Area'].replace('',0)
zipcodes2['Pop_Density(per Sqm)'] = (zipcodes2['Current Pop'].astype(float)/ zipcodes2['Area'].astype(float)).round()

In [ ]:
zipcodes2.sample(20)

In [ ]:
zipcodes2[['Official Zip Code Name', 'Zip Code State', 'Zip Code Type', 'County','Current Pop',
           'Racial_Majority', 'Unemployment_Rate','Median Household Income',
           'School_Test_Performance', 'Elevation Range', 'Area(Sqm)', 'Coordinates',
           'Pub_School_Racial_Majority','Alternate Zip Code Names', 
           'Lat', 'Lan', 'Pop_Density(per Sqm)']].sample(20)

In [ ]:
zipcodes2.to_csv('zipcodestable_final_part3.csv')

In [ ]:
End!